In [1]:
DATA_LOCATION = "/work/mk423/Anxiety/"
UMC_PATH = "/hpc/home/mk423/Anxiety/Universal-Mouse-Code/"
MODEL_PATH = "../Models/"

FLX_TRAIN_FILE = DATA_LOCATION + "FLX_train_dict_old_features.pkl"
FLX_VAL_FILE = DATA_LOCATION + "FLX_validation_dict_old_features.pkl"
FLX_TEST_FILE = DATA_LOCATION + "FLX_test_dict_old_features.pkl"

EPM_TRAIN_FILE = DATA_LOCATION + "EPM_train_dict_May_17.pkl"
EPM_VAL_FILE = DATA_LOCATION + "EPM_val_dict_May_17.pkl"
EPM_TEST_FILE = DATA_LOCATION + "EPM_test_dict_May_17.pkl"

OFT_TRAIN_FILE = DATA_LOCATION + "OFT_train_dict_old_features_hand_picked.pkl"
OFT_VAL_FILE = DATA_LOCATION + "OFT_validation_dict_old_features_hand_picked.pkl"
OFT_TEST_FILE = DATA_LOCATION + "OFT_test_dict_old_features_hand_picked.pkl"

FEATURE_LIST = ['X_power_1_2','X_coh_1_2','X_gc_1_2']
FEATURE_VECTOR = FEATURE_LIST
FEATURE_WEIGHT = [10,1,1]

MT_MODEL_PATH = MODEL_PATH + "Positive_MT_10_res_loss_10_power_features.pt"
PROJECTION_SAVE_PATH = "/hpc/home/mk423/Anxiety/MultiTaskWork/Projections/withinExperiment/final_projections/"

TRAIN = False
PROJ_TEST = True

In [2]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import sys
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
sys.path.append(UMC_PATH)
import umc_data_tools as umc_dt
from dCSFA_NMF import dCSFA_NMF

if torch.cuda.is_available():
    device="cuda:0"
else:
    device="cpu"

print("Using device: %s"%(device))

#For Consistency
RANDOM_STATE=42


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


### Load OFT Data

In [3]:
with open(OFT_TRAIN_FILE,'rb') as f:
    train_dict = pickle.load(f)

with open(OFT_VAL_FILE,'rb') as f:
    val_dict = pickle.load(f)
    
with open(OFT_TEST_FILE,'rb') as f:
    test_dict = pickle.load(f)
    

running_idx = 0
feature_groups = []
for idx,feature in enumerate(FEATURE_LIST):
    f_begin = running_idx
    f_end = f_begin + train_dict[feature].shape[1] 
    if idx == 0:
        f_end = f_end -1
    feature_groups.append((f_begin,f_end))

    running_idx = f_end

NUM_FREQS = 56
NUM_FEATURES = np.hstack([train_dict[feature] for feature in FEATURE_LIST]).shape[1] // NUM_FREQS
scale_vector = np.array([np.arange(1,NUM_FREQS+1) for feature in range(NUM_FEATURES)]).flatten()

#Train Arrays
oft_X_train = np.hstack([train_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])#*scale_vector
#oft_X_train[np.isnan(oft_X_train)] = 0
#oft_X_train[oft_X_train<0] = 0
oft_y_hc_train = train_dict['y_Homecage'].astype(bool)
oft_y_task_train = ~oft_y_hc_train
oft_y_ROI_train = train_dict['y_ROI']
oft_y_vel_train = train_dict['y_vel']
oft_y_mouse_train = train_dict['y_mouse']
oft_y_time_train = train_dict['y_time']

#Validation Arrays
oft_X_val = np.hstack([val_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])#*scale_vector
oft_y_hc_val = val_dict['y_Homecage'].astype(bool)
oft_y_task_val = ~oft_y_hc_val
oft_y_ROI_val = val_dict['y_ROI']
oft_y_vel_val = val_dict['y_vel']
oft_y_mouse_val = val_dict['y_mouse']
oft_y_time_val = val_dict['y_time']

#Test Arrays
oft_X_test = np.hstack([test_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])
oft_y_hc_test = test_dict['y_Homecage'].astype(bool)
oft_y_task_test = ~oft_y_hc_test
oft_y_ROI_test = test_dict['y_ROI']
oft_y_vel_test = test_dict['y_vel']
oft_y_mouse_test = test_dict['y_mouse']
oft_y_time_test = test_dict['y_time']

oft_X = np.vstack([oft_X_train,oft_X_val])
oft_y_task = np.hstack([oft_y_task_train,oft_y_task_val])
oft_y_mouse = np.hstack([oft_y_mouse_train,oft_y_mouse_val])

In [7]:
for mouse in np.unique(oft_y_mouse):
    mouse_mask = (oft_y_mouse==mouse)
    
    if len(np.unique(oft_y_task[mouse_mask==1])) < 2:
        print(mouse)
        print(np.unique(oft_y_task[mouse_mask==1]))
print(len(np.unique(oft_y_mouse)))

Mouse04215
[ True]
Mouse3192
[ True]
Mouse3194
[ True]
Mouse3203
[ True]
19


## Load FLX Data

In [45]:
with open(FLX_TRAIN_FILE,"rb") as f:
    flx_train_dict = pickle.load(f)

with open(FLX_VAL_FILE,"rb") as f:
    flx_validation_dict = pickle.load(f)
    
with open(FLX_TEST_FILE,"rb") as f:
    flx_test_dict = pickle.load(f)

flx_X_train = np.hstack([flx_train_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])
flx_y_train = flx_train_dict['y_flx']
flx_y_mouse_train = flx_train_dict['y_mouse']
flx_y_expDate_train = flx_train_dict['y_expDate']
flx_y_time_train = flx_train_dict['y_time']

flx_X_validation = np.hstack([flx_validation_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])
flx_y_validation = flx_validation_dict['y_flx']
flx_y_mouse_validation = flx_validation_dict['y_mouse']
flx_y_expDate_validation = flx_validation_dict['y_expDate']
flx_y_time_validation = flx_validation_dict['y_time']

flx_X_test = np.hstack([flx_test_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])
flx_y_test = flx_test_dict['y_flx']
flx_y_mouse_test = flx_test_dict['y_mouse']
flx_y_expDate_test = flx_test_dict['y_expDate']
flx_y_time_test = flx_test_dict['y_time']

flx_X = np.vstack([flx_X_train,flx_X_validation])
flx_y_task = np.hstack([flx_y_train,flx_y_validation])
flx_y_mouse = np.hstack([flx_y_mouse_train,flx_y_mouse_validation])
flx_y_expDate = np.hstack([flx_y_expDate_train,flx_y_expDate_validation])
flx_y_time = np.hstack([flx_y_time_train,flx_y_time_validation])

### Load EPM Data

In [46]:
with open(EPM_TRAIN_FILE,"rb") as f:
    epm_train_dict = pickle.load(f)

with open(EPM_VAL_FILE,"rb") as f:
    epm_validation_dict = pickle.load(f)
    
with open(EPM_TEST_FILE,"rb") as f:
    epm_test_dict = pickle.load(f)

#Load the data
NUM_FREQS = 56
NUM_FEATURES = (epm_train_dict["X_power_1_2"].shape[1] + \
                epm_train_dict["X_coh_1_2"].shape[1] + \
                epm_train_dict["X_gc_1_2"].shape[1]) // NUM_FREQS
SCALE_VECTOR = np.array([np.arange(1,57) for feature in range(NUM_FEATURES)]).flatten()

X_train = np.hstack([epm_train_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])
X_train[X_train<0] = 0
y_train = (epm_train_dict['y_ROI']%2).astype(bool)
y_in_task_mask_train = ~epm_train_dict['y_Homecage'].astype(bool)
y_mouse_train = epm_train_dict['y_mouse']
y_time_train = epm_train_dict['y_time']
train_nan_mask = (epm_train_dict['y_ROI'] > 0)


X_train_task = X_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_train_task = y_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_mouse_train_task = y_mouse_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_time_train_task = y_time_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
X_val = np.hstack([epm_validation_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])


y_val = (epm_validation_dict['y_ROI']%2).astype(bool)
y_in_task_mask_val= ~epm_validation_dict['y_Homecage'].astype(bool)
y_mouse_val = epm_validation_dict['y_mouse']
y_time_val = epm_validation_dict['y_time']
val_nan_mask = (epm_validation_dict['y_ROI'] > 0)

X_val_task = X_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_val_task = y_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_mouse_val_task = y_mouse_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_time_val_task = y_time_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]

X_test = np.hstack([epm_test_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])
y_test = (epm_test_dict['y_ROI']%2).astype(bool)
y_in_task_mask_test= ~epm_test_dict['y_Homecage'].astype(bool)
y_mouse_test = epm_test_dict['y_mouse']
y_time_test = epm_test_dict['y_time']
test_nan_mask = (epm_test_dict['y_ROI'] > 0)

X_test_task = X_test[np.logical_and(y_in_task_mask_test==1,test_nan_mask)==1]
y_test_task = y_test[np.logical_and(y_in_task_mask_test==1,test_nan_mask)==1]
y_mouse_test_task = y_mouse_test[np.logical_and(y_in_task_mask_test==1,test_nan_mask)==1]
y_time_test_task = y_time_test[np.logical_and(y_in_task_mask_test==1,test_nan_mask)==1]

epm_X = np.vstack([X_train_task,X_val_task])
epm_y_task = np.hstack([y_train_task,y_val_task])
epm_y_mouse = np.hstack([y_mouse_train_task,y_mouse_val_task])
epm_y_time = np.hstack([y_time_train_task,y_time_val_task])


### Concatenate Experiment Data

In [47]:
mt_X_train = np.vstack([flx_X_train,oft_X_train,X_train])
mt_y_train = np.hstack([flx_y_train,oft_y_task_train,y_in_task_mask_train])
mt_y_mouse_train = np.hstack([flx_y_mouse_train,oft_y_mouse_train,y_mouse_train])

mt_X_val = np.vstack([flx_X_validation,oft_X_val,X_val])
mt_y_val = np.hstack([flx_y_validation,oft_y_task_val,y_in_task_mask_val])
mt_y_mouse_val = np.hstack([flx_y_mouse_validation,oft_y_mouse_val,y_mouse_val])

mt_X_test = np.vstack([flx_X_test,oft_X_test,X_test])
mt_y_test = np.hstack([flx_y_test,oft_y_task_test,y_in_task_mask_test])
mt_y_mouse_test = np.hstack([flx_y_mouse_test,oft_y_mouse_test,y_mouse_test])

intercept_mask = OneHotEncoder().fit_transform(mt_y_mouse_train.reshape(-1,1)).todense()

### Load or Train the Model

In [9]:
if TRAIN:
    dummy_path = MODEL_PATH + "dummy_model.pt"
    N_COMPONENTS=20
    DIM_IN = flx_X_train.shape[1]
    NETWORK_CONSTRAINT = "Positive"
    model = dCSFA_NMF(n_components=N_COMPONENTS,dim_in=DIM_IN,device='auto',
                    n_intercepts=intercept_mask.shape[1],
                    n_sup_networks=1,optim_name='SGD',recon_loss='MSE',
                    sup_recon_weight=1,sup_weight=1,
                    useDeepEnc=True,h=256,sup_recon_type="Residual",
                    feature_groups=feature_groups,fixed_corr=NETWORK_CONSTRAINT)
    
    model.fit(mt_X_train,mt_y_train,intercept_mask=intercept_mask,batch_size=128,lr=1e-3,
            n_pre_epochs=100,n_epochs=1500,verbose=True,pretrain=True,best_model_name=dummy_model,
            momentum=0.9)
    
    torch.save(model,HIGH_WEIGHT_SCALED_POWER.format(NETWORK_CONSTRAINT))
    model.eval()
else:
    model = torch.load(MT_MODEL_PATH,map_location='cpu')
    model.device='cpu'
    model.eval()

## Multitask Performance Evaluation

In [15]:
y_pred, s = model.predict(mt_X_train,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,mt_y_train,mt_y_mouse_train)
mw_auc_dict = umc_dt.lpne_auc(y_pred,mt_y_train,mt_y_mouse_train,s,True)
print("\nTraining Mann Whitney AUCs: ")
for key in mw_auc_dict.keys():
    print(key,mw_auc_dict[key])

y_pred, s = model.predict(mt_X_val,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,mt_y_val,mt_y_mouse_val)
mw_auc_dict = umc_dt.lpne_auc(y_pred,mt_y_val,mt_y_mouse_val,s,True)
print("\nValidation MW AUCs: ")
for key in mw_auc_dict.keys():
    print(key,mw_auc_dict[key])

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning,


Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0

Training Mann Whitney AUCs: 
auc_method mannWhitneyU
Mouse04191 (1.0, 2.3405065517684796e-76)
Mouse04201 (0.9959099342216226, 3.8487378126022135e-211)
Mouse04202 (1.0, 3.6232340243012323e-212)
Mouse04205 (1.0, 1.6881996393705464e-126)
Mouse04215 (0.9951033564894951, 3.864022036343376e-145)
Mouse0630 (0.9980808744537272, 2.423105940812409e-83)
Mouse0634 (0.9951647411210954, 1.3239276915837364e-66)
Mouse0643 (0.9901684114701866, 7.804621790329968e-48)
Mouse1551 (0.9994023829977963, 3.0054331652999895e-69)
Mouse3191 (0.9969137412392133, 0.0)
Mouse3192 (nan, nan)
Mouse3193 (0.9992816311298831, 0.0)
Mouse3194 (0.9995378538552278, 0.0)
Mouse3202 (nan, nan)
Mouse32

In [18]:
y_pred, s = model.predict(mt_X_train,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,mt_y_train,mt_y_mouse_train)
print("train roc auc mean: %0.2f +/- %0.2f"%(roc_mean,roc_std))
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,mt_y_train,mt_y_mouse_train,s,True)
print("train mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
y_pred, s = model.predict(mt_X_val,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,mt_y_val,mt_y_mouse_val)
print("val roc auc mean: %0.2f +/- %0.2f"%(roc_mean,roc_std))
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,mt_y_val,mt_y_mouse_val,s,True)
print("val mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning,


train roc auc mean: 0.99 +/- 0.00
Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
train mw auc mean: 1.00 +/- 0.00
val roc auc mean: 0.67 +/- 0.02
val mw auc mean: 0.76 +/- 0.02


## EPM Performance

In [49]:
training_save_file = PROJECTION_SAVE_PATH + "MT_model_EPM_data_train_projections_2022_Oct_4.csv"
validation_save_file = PROJECTION_SAVE_PATH + "MT_model_EPM_data_val_projections_2022_Oct_4.csv"
testing_save_file = PROJECTION_SAVE_PATH + "MT_model_EPM_data_test_projections_2022_Oct_4.csv"

y_pred, s = model.predict(X_train,include_scores=True)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_in_task_mask_train,y_mouse_train,s,True)
print("train mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
mw_auc_dict = umc_dt.lpne_auc(y_pred,y_in_task_mask_train,y_mouse_train,s,True)
df = umc_dt.make_projection_csv(EPM_TRAIN_FILE,model,FEATURE_VECTOR,
                                ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel','y_locx','y_locy'],
                                save_file =training_save_file,
                                auc_dict=mw_auc_dict,weights=FEATURE_WEIGHT)

y_pred, s = model.predict(X_val,include_scores=True)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_in_task_mask_val,y_mouse_val,s,True)
print("val mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
mw_auc_dict = umc_dt.lpne_auc(y_pred,y_in_task_mask_val,y_mouse_val,s,True)
df = umc_dt.make_projection_csv(EPM_VAL_FILE,model,FEATURE_VECTOR,
                                ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel','y_locx','y_locy'],
                                save_file =validation_save_file,
                                auc_dict=mw_auc_dict,weights=FEATURE_WEIGHT)

if PROJ_TEST:
    y_pred, s = model.predict(X_test,include_scores=True)
    mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_in_task_mask_test,y_mouse_test,s,True)
    print("test mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
    mw_auc_dict = umc_dt.lpne_auc(y_pred,y_in_task_mask_test,y_mouse_test,s,True)
    
    df = umc_dt.make_projection_csv(EPM_TEST_FILE,model,FEATURE_VECTOR,
                                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel','y_locx','y_locy'],
                                    save_file=testing_save_file,
                                    auc_dict=mw_auc_dict,weights=FEATURE_WEIGHT)

train mw auc mean: 1.00 +/- 0.00
val mw auc mean: 0.74 +/- 0.02
test mw auc mean: 0.73 +/- 0.02


## FLX Performance

In [50]:
#We want to project for the complete data recording
FULL_FEATURE_LIST = ["X_power_1_2_full","X_coh_1_2_full","X_gc_1_2_full"]
training_save_file = PROJECTION_SAVE_PATH + "MT_model_FLX_data_train_projections_2022_Oct_4.csv"
validation_save_file = PROJECTION_SAVE_PATH + "MT_model_FLX_data_val_projections_2022_Oct_4.csv"
testing_save_file = PROJECTION_SAVE_PATH + "MT_model_FLX_data_test_projections_2022_Oct_4.csv"

y_pred, s = model.predict(flx_X_train,include_scores=True)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_train,flx_y_mouse_train,s,True)
print("train mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
mw_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_train,flx_y_mouse_train,s,True)

df = umc_dt.make_projection_csv(FLX_TRAIN_FILE,model,FULL_FEATURE_LIST,
                    ['y_mouse_full','y_expDate_full','y_time_full','y_flx_full'],
                    save_file = training_save_file,
                    auc_dict=mw_auc_dict,mousevar="y_mouse_full",
                    weights=FEATURE_WEIGHT)

y_pred, s = model.predict(flx_X_validation,include_scores=True)
mw_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_validation,flx_y_mouse_validation,s,True)
print("Validation Mouse AUCs: ")
for key in mw_auc_dict.keys():
    print(key,mw_auc_dict[key])

df = umc_dt.make_projection_csv(FLX_VAL_FILE,model,FULL_FEATURE_LIST,
                ['y_mouse_full','y_expDate_full','y_time_full','y_flx_full'],
                save_file = validation_save_file,
                auc_dict=mw_auc_dict,mousevar="y_mouse_full",
                weights=FEATURE_WEIGHT)

if PROJ_TEST:
    y_pred, s = model.predict(flx_X_test,include_scores=True)
    mw_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_test,flx_y_mouse_test,s,True)
    print("Test Mouse AUCs: ")
    for key in mw_auc_dict.keys():
        print(key,mw_auc_dict[key])
    df = umc_dt.make_projection_csv(FLX_TEST_FILE,model,FULL_FEATURE_LIST,
                    ['y_mouse_full','y_expDate_full','y_time_full','y_flx_full'],
                    save_file = testing_save_file,
                    auc_dict=mw_auc_dict,mousevar="y_mouse_full",weights=FEATURE_WEIGHT)

Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
train mw auc mean: 1.00 +/- 0.00
Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
Validation Mouse AUCs: 
auc_method mannWhitneyU
Mouse3192 (0.7102833553986466, 2.5857582637996656e-105)
Mouse  Mouse99002  has only one class - AUC cannot be calculated
n_positive samples  1890
n_negative samples  0
Mouse  Mouse99021  has only one class - AUC cannot be calculated
n_positive samples  2025
n_negative samples  0
Test Mouse AUCs: 
auc_method mannWhitneyU
Mouse69061 (0.4336935712065354, 2.697813125552194e-12)
Mouse99002 (nan, nan)
Mouse99021 (nan, nan)


## OFT Performance

In [51]:
training_save_file = PROJECTION_SAVE_PATH + "MT_model_OFT_data_train_projections_2022_Oct_4.csv"
validation_save_file = PROJECTION_SAVE_PATH + "MT_model_OFT_data_val_projections_2022_Oct_4.csv"
testing_save_file = PROJECTION_SAVE_PATH + "MT_model_OFT_data_test_projections_2022_Oct_4.csv"
y_pred, s = model.predict(oft_X_train,include_scores=True)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task_train.astype(bool),oft_y_mouse_train,s,True)
print("train mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
mw_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task_train,oft_y_mouse_train,s,True)

df = umc_dt.make_projection_csv(OFT_TRAIN_FILE,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = training_save_file,
                    auc_dict=mw_auc_dict,
                    weights=FEATURE_WEIGHT)

y_pred, s = model.predict(oft_X_val,include_scores=True)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task_val.astype(bool),oft_y_mouse_val,s,True)
print("val mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
mw_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task_val,oft_y_mouse_val,s,True)

df = umc_dt.make_projection_csv(OFT_VAL_FILE,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = validation_save_file,
                    auc_dict=mw_auc_dict,
                    weights=FEATURE_WEIGHT)

if PROJ_TEST:
    y_pred, s = model.predict(oft_X_test,include_scores=True)
    mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task_test.astype(bool),oft_y_mouse_test,s,True)
    print("test mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
    mw_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task_test,oft_y_mouse_test,s,True)
    df = umc_dt.make_projection_csv(OFT_TEST_FILE,model,FEATURE_VECTOR,
                        ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                        save_file = testing_save_file,
                        auc_dict=mw_auc_dict,
                        weights=FEATURE_WEIGHT)
    


Mouse  Mouse04215  has only one class - AUC cannot be calculated
n_positive samples  506
n_negative samples  0
Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3194  has only one class - AUC cannot be calculated
n_positive samples  324
n_negative samples  0
Mouse  Mouse3203  has only one class - AUC cannot be calculated
n_positive samples  500
n_negative samples  0
train mw auc mean: 1.00 +/- 0.00
Mouse  Mouse04215  has only one class - AUC cannot be calculated
n_positive samples  506
n_negative samples  0
Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3194  has only one class - AUC cannot be calculated
n_positive samples  324
n_negative samples  0
Mouse  Mouse3203  has only one class - AUC cannot be calculated
n_positive samples  500
n_negative samples  0
val mw auc mean: 0.82 +/- 0.06
test mw auc mean: 0.81 +/- 0.03
